#### Sparse Image Warp   
Image warping using correspondences between sparse control points.

Apply a non-linear warp to the image, where the warp is specified by the source and destination locations of a (potentially small) number of control points. First, we use a polyharmonic spline (```tf.contrib.image.interpolate_spline```) to interpolate the displacements between the corresponding control points to a dense flow field. Then, we warp the image using this dense flow field (```tf.contrib.image.dense_image_warp```).

Let t index our control points. For regularization_weight=0, we have: warped_image[b, dest_control-point_location[b, t, 0], dest_control_point_location[b, t, 1], :] = image[b, source_control_point_location[b, t, 0], source_control_point_location[b, t, 1], :]

For regularization_weight > 0, this condition is met approximately, since regularized interpolation trades off smoothness of the interpolant vs. reconstruction of the interpolant at the control points. See ```tf.contrib.image.interpolate_spline``` for further documentation of the interpolation_order and regularization_weight arguments.

| Args||   
|:------------------------------ |:--------------------------------------------|
|image                           |[batch, height, width, channels] float Tensor|
|source_control_point_locations  |[batch, num_control_points, 2]   float Tensor|
|dest_control_point_locations    |[batch, num_control_points, 2]   float Tensor|
|interpolation_order             |polynomial order used by the spline interpolation|
|regularization_weight           |weight on smoothness regularizer in interpolation|

| Returns||   
|:------------------------------ |:--------------------------------------------|
|warped_image                    |[batch, height, width, channels] float Tensor with same type as input image.|
|flow_field                      |[batch, height, width, 2] float Tensor containing the dense flow field produced by the interpolation.|


#### Import Library

In [1]:
import numpy as np
import scipy as sp
import random

#### Input Data shape

image.shape = [```batch_size```, ```image_height```, ```image_width```, ```channels```]

#### My Data

In [2]:
image = np.ones((800, 2, 257, 382))
image = np.transpose(image, (0, 2, 3, 1))
print("input image shape :", image.shape)

input image shape : (800, 257, 382, 2)


In [3]:
batch_size, image_height, image_width, _ = list(image.shape)
print("batch size   :", batch_size)
print("image_height :", image_height)
print("image_width  :", image_width)

batch size   : 800
image_height : 257
image_width  : 382


#### Define Length Parameter

In [4]:
spec = image
num_rows = spec.shape[1]
spec_len = spec.shape[2]
print("num_rows :", num_rows)
print("spec_len :", spec_len)

num_rows : 257
spec_len : 382


#### Horizontal Line

In [5]:
y = num_rows // 2
horizontal_line_at_ctr = spec[0][y]

print("y :", y)
print("horizontal_line_at_ctr's shape :", horizontal_line_at_ctr.shape)

y : 128
horizontal_line_at_ctr's shape : (382, 2)


&nbsp;```y: 128```&nbsp; : &nbsp;middle of height   
```(382, 2)``` : ```(spec_len, channels)```

#### Get Random Points & Distance

In [6]:
W = 5

point_a = random.randrange(W, spec_len-W)
point_to_warp = horizontal_line_at_ctr[point_a]

print("random range : ", point_a)
print(point_to_warp)

random range :  135
[1. 1.]


In [7]:
dist_to_warp = random.randrange(-W, W)
print("dist_to_warp :", dist_to_warp)

dist_to_warp : -5


```dist_to_warp``` : distance to warp

#### Define Input Parameter 1 at Sparse Image Warp

In [8]:
src_pts = np.array([[[y, point_to_warp]]])
dest_pts = np.array([[[y, point_to_warp + dist_to_warp]]])

# src_pts = torch.tensor([[[y, point_to_warp]]])
# dest_pts = torch.tensor([[[y, point_to_warp + dist_to_warp]]])

print(src_pts)
print(dest_pts)

[[[128 array([1., 1.])]]]
[[[128 array([-4., -4.])]]]


In [9]:
source_control_point_locations = src_pts
dest_control_point_locations = dest_pts

```source_control_point_locations``` = ```src_pts```   
```dest_control_point_locations``` = ```dest_pts```

#### Control Points Flows

In [10]:
control_point_flows = (dest_control_point_locations - source_control_point_locations)
print("control_point_flows :", control_point_flows)

control_points_flows : [[[0 array([-5., -5.])]]]


#### Get boundary Location & Condition

##### Sparse image warp --> Add zero flow controls at boundary --> get boundary locations

```num_points_per_edge``` =  ```boundary_points_per_edge```
```control_point_locations``` = ```dest_control_point_locations```

In [21]:
boundary_points_per_edge = 0
control_point_locations = dest_control_point_locations

In [12]:
def _get_boundary_locations(image_height, image_width, boundary_points_per_edge):
    y_range = np.linspace(0, image_height - 1, boundary_points_per_edge + 2)
    x_range = np.linspace(0, image_width - 1, boundary_points_per_edge + 2)
    ys, xs = np.meshgrid(y_range, x_range, indexing='ij')
    
    is_boundary = np.logical_or(
        np.logical_or(xs == 0, xs == image_width - 1),
        np.logical_or(ys == 0, ys == image_height - 1)
    )
    return np.stack([ys[is_boundary], xs[is_boundary]], axis=-1)

In [13]:
y_range = np.linspace(0, image_height - 1, boundary_points_per_edge + 2)
x_range = np.linspace(0, image_width - 1, boundary_points_per_edge + 2)
ys, xs = np.meshgrid(y_range, x_range, indexing='ij')

is_boundary = np.logical_or(
    np.logical_or(xs == 0, xs == image_width - 1),
    np.logical_or(ys == 0, ys == image_height - 1)
)

boundary_point_locations = np.stack([ys[is_boundary], xs[is_boundary]], axis=-1)

print("y_range :", y_range)
print("x_range :", x_range)
print("ys :\n", ys)
print("xs :\n", xs)
print("boundary_point_locations : \n", boundary_point_locations)

y_range : [  0. 256.]
x_range : [  0. 381.]
ys :
 [[  0.   0.]
 [256. 256.]]
xs :
 [[  0. 381.]
 [  0. 381.]]
boundary_point_locations : 
 [[  0.   0.]
 [  0. 381.]
 [256.   0.]
 [256. 381.]]


#### Boundary Flows to Zero

In [14]:
boundary_point_flows = np.zeros([boundary_point_locations.shape[0], 2])
print("boundary_point_flows :", boundary_point_flows)

boundary_point_flows : [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


```boundary_point_locations.shape[0]``` = number of boundary points

In [15]:
type_to_use = dest_control_point_locations.dtype
print(type_to_use)

object


In [16]:
dest_control_point_locations

array([[[128, array([-4., -4.])]]], dtype=object)

#### Expand Boundary Points To minibatch

In [19]:
def _expand_to_minibatch(np_array, batch_size):
    tiles = [batch_size] + [1] * np_array.ndim
    return np.tile(np.expand_dims(np_array, 0), tiles)

boundary_point_locations = _expand_to_minibatch(boundary_point_locations, batch_size)
boundary_point_flows = _expand_to_minibatch(boundary_point_flows, batch_size)
print("boundary_point_locations.shape : ", boundary_point_locations.shape)
print("boundary_point_flows.shape : ", boundary_point_flows.shape)

boundary_point_locations.shape :  (800, 4, 2)
boundary_point_flows.shape :  (800, 4, 2)


#### Merge Control Point Locations

In [ ]:
merged_control_point_location = np.concatenate([control_point_locations])